In [ ]:
from torch.utils.data import Dataset
from transformers import AutoTokenizer
from MTGpred.utils.mtgjson import simplify_name, parse_mana_cost
import re
import torch


class AllPickedCardsDataset(Dataset):
    def __init__(
        self,
        cards_df: pd.DataFrame,
        picked_cards: pd.DataFrame,
        model_name: str = "xlnet-base-cased",
        cased: bool = True,
        join_tokens: bool = True,
        max_length: int = 4096,
        truncation: bool = True,
        include_name: bool = True,
    ):
        """
        Dataset for decks classification

        Parameters
        ----------
        cards_df : pd.DataFrame
            Dataframe with all the cards
        picked_cards : pd.DataFrame
            Dataframe with all the picked cards
        model_name : str, optional
            Name of the transformer model, by default "xlnet-base-cased"
        cased : bool, optional
            Cased the card output or not, by default True
        join_tokens : bool, optional
            Join all texts and tokenize all or not, by default False
        max_length : int, optional
            Max length of the tokenizer, by default 256
        truncation : bool, optional
            Truncate the text or not, by default True
        """

        assert not join_tokens or (
            join_tokens and truncation
        ), "If join_tokens is True, truncation must be True"

        self.cards_df = cards_df
        self.data = picked_cards
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.cased = cased
        self.join_tokens = join_tokens
        self.max_length = max_length
        self.truncation = truncation
        self.include_name = include_name

    def __len__(self):
        return len(self.data)

    def preprocess_card(self, name: str):
        simplified_name = simplify_name(name)
        all_variations = []

        selected_card = self.cards_df[
            (self.cards_df["faceName"] == simplified_name)
            | (self.cards_df["name"] == simplified_name)
        ]
        if len(selected_card) == 0:
            print(
                f"WARNING: {name} cant be found in the database. Will be removed from the deck."
            )
            return []

        for index, variations in selected_card.iterrows():
            mana_cost = (
                parse_mana_cost(variations["manaCost"])
                if not pd.isna(variations["manaCost"])
                else ""
            )

            card_type = variations["type"]

            text = variations["text"] if not pd.isna(variations["text"]) else ""
            mana_in_text = re.findall(r"\{.*\}", text)
            for mana in mana_in_text:
                text = text.replace(mana, parse_mana_cost(mana))

            stats = f"{variations['power']} power, {variations['power']} power"

            if self.include_name:
                input_text = ". ".join([name, mana_cost, card_type, text, stats])
            else:
                input_text = ". ".join([mana_cost, card_type, text, stats])

            if not self.cased:
                input_text = input_text.lower()

            all_variations.append(input_text)

        return all_variations

    def get_tokenized_text(self, picks_data):
        all_cards = []

        for card in picks_data:
            all_cards.extend(self.preprocess_card(card))

        if self.join_tokens:
            all_cards = self.tokenizer.sep_token.join(all_cards)

            tokenized_deck = self.tokenizer(
                all_cards,
                padding="max_length",
                max_length=self.max_length,
                truncation=self.truncation,
            )
        else:
            tokenized_deck = []
            for card in all_cards:
                tokenized_deck.append(
                    self.tokenizer(
                        card,
                        return_tensors="pt",
                        padding="max_length",
                        max_length=self.max_length,
                        truncation=self.truncation,
                    )
                )

        return tokenized_deck

    def __getitem__(self, idx):
        picks_data = self.data.iloc[idx]

        picks = self.get_tokenized_text(picks_data["picks"])

        if self.join_tokens:
            picks["labels"] = torch.tensor(picks_data["wins"])

            return picks
        else:
            return {
                "input": picks,
                "label": picks_data["wins"],
                "mask_length": len(picks),
            }


In [1]:
import os
import pandas as pd


def get_drafted_data(data_folder: str, num_drafts: int = 10000):
    files = os.listdir(data_folder)
    total_drafts = 0
    drafts = []

    for f in files:
        df = pd.read_csv(os.path.join(data_folder, f))
        groups = df.groupby("draft_id")

        for draft_id, group in groups:
            drafts.append(
                pd.DataFrame(
                    {
                        "picks": [list(group["pick"])],
                        "wins": group["event_match_wins"].iloc[0],
                    }
                )
            )

        total_drafts += len(df["draft_id"].unique())

        if total_drafts >= num_drafts:
            break

    return pd.concat(drafts)


df = get_drafted_data(
    "C:/Users/javij/OneDrive/Escritorio/Proyectos/MTG_predictions/data/draft/draft_data/"
)

df

<bound method NDFrame.describe of                                                 picks  wins
0   [Trostani, Three Whispers, Shock, Dog Walker, ...     4
0   [Commercial District, Outrageous Robbery, Tunn...     6
0   [Leering Onlooker, Deadly Cover-Up, Harried Dr...     1
0   [Surveillance Monitor, Cold Case Cracker, Ceas...     6
0   [Fugitive Codebreaker, Galvanize, Shock, Mistw...     4
..                                                ...   ...
0   [Neighborhood Guardian, Seasoned Consultant, F...     1
0   [Soul Enervation, Nightdrinker Moroii, Surveil...     2
0   [Evidence Examiner, Axebane Ferox, Repulsive M...     1
0   [Doppelgang, Hard Evidence, Tunnel Tipster, Vi...     7
0   [No Witnesses, Wojek Investigator, Museum Nigh...     2

[10738 rows x 2 columns]>

In [3]:
df["wins"].value_counts()

1    2051
2    1926
0    1598
3    1566
4    1165
7    1061
5     806
6     565
Name: wins, dtype: int64

In [ ]:
import json
from MTGpred.utils.mtgjson import load_cards_df
from sklearn.model_selection import train_test_split

cards_df = load_cards_df(data_path="../data/AtomicCards.json")
train_cards, test_cards = train_test_split(df, test_size=0.2)

train_dataset = AllPickedCardsDataset(
    cards_df, train_cards, model_name="allenai/longformer-base-4096", cased=False
)
test_dataset = AllPickedCardsDataset(
    cards_df, test_cards, model_name="allenai/longformer-base-4096", cased=False
)


In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import evaluate
import numpy as np

accuracy = evaluate.load("accuracy")


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)


model = AutoModelForSequenceClassification.from_pretrained(
    "allenai/longformer-base-4096", num_labels=3
)

training_args = TrainingArguments(
    output_dir="models/longformer",
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,
    per_device_eval_batch_size=1,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    gradient_checkpointing=True,
    logging_steps=5,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)